# Session 2 – RAG Evaluering med ragas

Evaluer minimal RAG-pipeline ved hjælp af ragas-metrics: answer_relevancy, faithfulness, context_precision.


# Scenario
Dette scenario evaluerer en minimal Retrieval Augmented Generation (RAG)-pipeline lokalt. Vi:
- Definerer et lille syntetisk dokumentkorpus.
- Indlejrer dokumenter og implementerer en simpel similarity retriever.
- Genererer faktabaserede svar ved hjælp af en lokal model (Foundry Local / OpenAI-kompatibel).
- Beregner ragas-metrics (`answer_relevancy`, `faithfulness`, `context_precision`).
- Understøtter en HURTIG tilstand (miljøvariabel `RAG_FAST=1`) for kun at beregne svarrelevans til hurtig iteration.

Brug denne notebook til at validere, at din lokale model + embeddings-stak producerer faktuelt baserede svar, før du skalerer til større korpusser.


### Forklaring: Installation af afhængigheder
Installerer nødvendige biblioteker:
- `foundry-local-sdk` til lokal modelhåndtering.
- `openai` klientinterface.
- `sentence-transformers` til tætte indlejringer.
- `ragas` + `datasets` til evaluering og beregning af metrikker.
- `langchain-openai` adapter til ragas LLM-interface.

Kan sikkert køres igen; spring over, hvis miljøet allerede er forberedt.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Forklaring: Kerneimporter & Metrics
Indlæser kernebiblioteker og ragas-metrics. Vigtige elementer:
- SentenceTransformer til embeddings.
- `evaluate` + udvalgte ragas-metrics.
- `Dataset` til opbygning af evalueringskorpus.
Disse importer udløser ikke eksterne kald (bortset fra mulig model-cache-indlæsning til embeddings).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Forklaring: Legetøjs-korpus & QA Grundsandhed
Definerer et miniature in-memory korpus (`DOCS`), et sæt af bruger-spørgsmål og forventede grundsandhedssvar. Disse muliggør hurtig, deterministisk beregning af metrikker uden eksterne dataindhentninger. I virkelige scenarier ville du udvælge produktionsforespørgsler + kuraterede svar.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Forklaring: Service Init, Embeddings & Safety Patch
Initialiserer Foundry Local manager, anvender en sikkerhedspatch for schema-drift til `promptTemplate`, løser model-id, opretter en OpenAI-kompatibel klient og forudberegner tætte embeddings for dokumentkorpusset. Dette opretter en genanvendelig tilstand til hentning + generering.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Forklaring: Retriever-funktion
Definerer en simpel vektorsimilaritets-retriever, der bruger prikprodukt på normaliserede embeddings. Returnerer de top-k dokumenter (standard k=2). I produktion bør den erstattes med en ANN-indeks (FAISS, Chroma, Milvus) for bedre skalerbarhed og lavere ventetid.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Forklaring: Genereringsfunktion
`generate` opbygger en begrænset prompt (systemet instruerer i KUN at bruge konteksten) og kalder den lokale model. Lav temperatur (0.1) fremmer troværdig udtrækning frem for kreativitet. Returnerer beskåret svartekst.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Forklaring: Fallback-klientinitialisering
Sikrer, at `client` eksisterer, selv hvis en tidligere initialiseringscelle blev sprunget over eller mislykkedes—forhindrer NameError under senere evalueringsskridt.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Forklaring: Evalueringsloop & Metrics
Bygger evalueringsdatasættet (påkrævede kolonner: spørgsmål, svar, kontekster, sandheder, reference) og itererer derefter udvalgte ragas-metrics.

Optimering:
- FAST_MODE begrænser sig til svarrelevans for hurtige testkørsler.
- Per-metric loop undgår fuld genberegning, hvis én metric fejler.

Returnerer en dict af metric -> score (NaN ved fejl).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
